In [2]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model



# Main Supervisor (The CEO)

In [7]:

# supervisor = create_supervisor(
#     model=init_chat_model("openai:gpt-4.1"),
#     supervisor_name= 'The CEO',
#     agents=[research_agent, math_agent],
#     prompt=(
#         "You are a supervisor managing two agents:\n"
#         "- a research agent. Assign research-related tasks to this agent\n"
#         "- a math agent. Assign math-related tasks to this agent\n"
#         "Assign work to one agent at a time, do not call agents in parallel.\n"
#         "Do not do any work yourself."
#     ),
#     add_handoff_back_messages=True,
#     output_mode="full_history",
# ).compile()

In [3]:
import json
import time
from sentence_transformers import SentenceTransformer, util

class PersistentRAG:
    def __init__(self, storage_file="rag_memory.json"):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")  # Small, fast embedding model
        self.memory = []
        self.storage_file = storage_file
        self.load_memory()

    def save(self, text, tags=None):
        """
        Save text, tags, timestamp, and embedding to memory.
        """
        embedding = self.model.encode(text).tolist()
        entry = {
            "text": text,
            "tags": tags or [],
            "timestamp": time.time(),
            "embedding": embedding
        }
        self.memory.append(entry)
        self.persist_memory()

    def retrieve(self, query, top_k=3, tag_filter=None):
        """
        Retrieve top_k most relevant texts based on embedding similarity.
        Optionally filter by tag.
        """
        query_embedding = self.model.encode(query)
        
        # Apply tag filter if provided
        candidates = [
            entry for entry in self.memory 
            if tag_filter is None or any(tag in entry["tags"] for tag in tag_filter)
        ]
        
        if not candidates:
            return []

        # Compute similarity scores
        embeddings = [entry["embedding"] for entry in candidates]
        scores = util.cos_sim(query_embedding, embeddings)[0]

        # Rank by score
        ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
        return ranked[:top_k]

    def persist_memory(self):
        """
        Save memory to disk as JSON.
        """
        with open(self.storage_file, "w") as f:
            json.dump(self.memory, f)

    def load_memory(self):
        """
        Load memory from disk if it exists.
        """
        try:
            with open(self.storage_file, "r") as f:
                self.memory = json.load(f)
        except FileNotFoundError:
            self.memory = []

/Users/siddharthmandgi/anaconda3/envs/AgenticAI-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Save some entries

rag = PersistentRAG()
rag.save("The capital of France is Paris.", tags=["geography"])
rag.save("Python is a popular programming language.", tags=["programming"])
rag.save("OpenAI develops artificial intelligence.", tags=["AI"])

# Retrieve with query
results = rag.retrieve("What is the capital of France?", top_k=2)

for entry, score in results:
    print(f"Retrieved: {entry['text']} (Score: {score:.4f})")

Retrieved: The capital of France is Paris. (Score: 0.8790)
Retrieved: The capital of France is Paris. (Score: 0.8790)
